## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bincore9_0_x = np.load('x_bc9_0.npy')
bincore9_1_x = np.load('x_bc9_1.npy')
print('x1완료')

bincore9_2_x = np.load('x_bc9_2.npy')
bincore9_3_x = np.load('x_bc9_3.npy')
print('x2완료')

bincore9_0_y = np.load('y_bc9_0.npy')
bincore9_1_y = np.load('y_bc9_1.npy')
bincore9_2_y = np.load('y_bc9_2.npy')
bincore9_3_y = np.load('y_bc9_3.npy')

x_bincore9 = np.concatenate((bincore9_0_x, bincore9_1_x, bincore9_2_x, bincore9_3_x), axis=0)
y_bincore9 = np.concatenate((bincore9_0_y, bincore9_1_y, bincore9_2_y, bincore9_3_y), axis=0)

print(x_bincore9.shape, y_bincore9.shape)

x1완료
x2완료
(342518, 192, 256) (342518, 192, 1)


In [2]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bincore9.shape[0])

x_bincore9 = x_bincore9[p]
y_bincore9 = y_bincore9[p]

print(x_bincore9.shape, y_bincore9.shape)

(342518, 192, 256) (342518, 192, 1)


## (7) 모델

In [3]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 10) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,192, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(288, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습

In [4]:

model1 = models.Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model1.summary()

import time
print('======Training stage======')
starttime = time.time()

model1.fit(x_bincore9,
           y_bincore9,
           epochs = 500,
           batch_size = 32,
           validation_split=0.3,
           callbacks=[early_stopping])

endtime = time.time()

print('걸린시간:', endtime - starttime)
model1.save('gcc9_bin_core_s192_h288_o0123.h5')
print('save 완료')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 192, 256)]        0         
_________________________________________________________________
bidirectional (Bidirectional (None, 192, 576)          1255680   
_________________________________________________________________
time_distributed (TimeDistri (None, 192, 1)            577       
Total params: 1,256,257
Trainable params: 1,256,257
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 239762 samples, validate on 102756 samples
Epoch 1/500
239762/239762 [==============================] - 312s 1ms/sample - loss: 0.0059 - accuracy: 0.9982 - val_loss: 0.0018 - val_accuracy: 0.9994
Epoch 2/500
239762/239762 [==============================] - 307s 1ms/sample - loss: 0.0012 - accuracy: 0.9996 - val_loss: 9.1020e-04 - val_accuracy:

걸린시간: 6040.624769210815
save 완료
